# Prompt Engineering Workshop


Outline:
1. Basic Prompting
2. Intermediate Techniques
3. Advanced with Tools

## Setup

Install the following packages:
- `openai` - for OpenAI API interactions
- `python-dotenv` - for loading environment variables from .env file
- `wikipedia` - for Wikipedia information retrieval

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
import wikipedia

# Load environment variables
load_dotenv(override=True)

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Helper function for simple API calls
def ask(prompt, model="gpt-4o-mini", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content

print("✅ Setup complete")

In [ ]:
# Try with various prompts
prompt = "Hi"
print(ask(prompt))

## Part 1: Basic Prompting Techniques

### 1.1 Zero-shot Prompting
Direct questions without examples

In [ ]:
# Zero-shot: Direct question
prompt = "Classify this text sentiment: 'The movie was absolutely fantastic!'"

result = ask(prompt)
print("Zero-shot result:")
print(result)

### 1.2 Few-shot Prompting
Provide examples to guide the model

In [ ]:
# Few-shot: With examples
prompt = """Classify the sentiment:

Text: "I love this product!"
Sentiment: Positive

Text: "This is terrible."
Sentiment: Negative

Text: "It's okay, nothing special."
Sentiment: Neutral

Text: "The movie was absolutely fantastic!"
Sentiment:"""

result = ask(prompt)
print("Few-shot result:")
print(result)

### 1.3 Role-based Prompting
Assign a specific role or persona

In [ ]:
# Without role
prompt1 = "Explain quantum computing"

result1 = ask(prompt1)
print("Without role:\n", result1)

In [ ]:
# With role
prompt2 = "You are a teacher explaining to a 10-year-old. Explain quantum computing"
result2 = ask(prompt2)
print("With role (teacher to 10-year-old):\n", result2)

### 1.4 Clear Instructions
Be specific about what you want

In [ ]:
# Vague instruction
vague = "Tell me about Paris. Where should I go?"
print("Vague:", ask(vague), "...\n")

In [ ]:
# Clear instruction
clear = "List exactly 3 tourist attractions in Paris with one sentence description each"
print("Clear:\n", ask(clear))

## Part 2: Intermediate Techniques

### 2.1 Chain-of-Thought (CoT) Prompting
Ask the model to think step-by-step

In [ ]:
# Without CoT
prompt1 = "Estimate how many taxis are actively carrying passengers in Singapore at 8:00 pm on a weekday. Provide a single numeric estimate."
print("Without CoT:")
print(ask(prompt1), "\n")

In [ ]:
# With CoT
prompt2 = """
Q:

A:

Q: Estimate how many taxis are actively carrying passengers in Singapore at 8:00 pm on a weekday. Provide a single numeric estimate. Think clearly step by step.

A:
""" # Input further prompt that encourages chain-of-thought reasoning
print("With CoT:")
print(ask(prompt2))

### 2.2 Step-by-Step Reasoning
Break complex tasks into steps

In [ ]:
prompt = """You are doing a Fermi estimate for Singapore at 8:00 pm on a weekday.
Reason step by step:
	1.
  2.
  3. """

result = ask(prompt)
print(result)

### 2.3 Output Formatting
Request specific output formats

In [ ]:
# JSON output
prompt = """Extract information from this text and return as JSON:

"John Smith, 28 years old, works as a software engineer in San Francisco."
"""


result = ask(prompt)
print(result)

try:
    data = json.loads(result)
    print("\n✅ Valid JSON parsed:", data)
except:
    print("\n❌ Invalid JSON")

In [ ]:
# List output
prompt = """Generate a numbered list of 5 healthy breakfast options.
Format: Number. Item - Brief description (max 10 words)"""

result = ask(prompt)
print("List Output:")
print(result)

## Part 3: Advanced with Tools

### 3.1 System vs User Prompts
Using system messages for consistent behavior

In [ ]:
def ask_with_system(system_prompt, user_prompt, temperature=0.7):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

# Example: Consistent pirate responses
system = "You are a pirate. Always respond in pirate speak."
user1 = "What's the weather like?"
user2 = "How do I cook pasta?"

print("Question 1:", user1)
print("Response:", ask_with_system(system, user1))
print("\nQuestion 2:", user2)
print("Response:", ask_with_system(system, user2))

### 3.2 Temperature Control
Controlling randomness in outputs

In [ ]:
prompt = "Write a one-sentence tagline for a coffee shop"

# Low temperature (0.1) - More deterministic
print("Low temperature (0.1) - Consistent:")
for i in range(3):
    print(f"{i+1}. {ask(prompt, temperature=0.1)}")

print("\nHigh temperature (1.0) - Creative:")
for i in range(3):
    print(f"{i+1}. {ask(prompt, temperature=1.0)}")

### 3.3 Wikipedia for Information Retrieval
Using Wikipedia API to get factual information

In [ ]:
# Using Wikipedia to get factual information
def get_article(search_term):
    """Search Wikipedia and retrieve article content"""
    try:
        # Search for articles
        results = wikipedia.search(search_term)
        if not results:
            return f"No Wikipedia articles found for '{search_term}'"

        # Get the first result
        first_result = results[0]
        page = wikipedia.page(first_result, auto_suggest=False)

        # Return the content (you can also access page.summary for a shorter version)
        return page.content
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle disambiguation pages
        return f"Multiple options found: {e.options[:5]}"
    except wikipedia.exceptions.PageError:
        return f"Page not found for '{search_term}'"
    except Exception as e:
        return f"Error: {str(e)}"

# Example: Get information from Wikipedia
print("Searching Wikipedia for information...\n")
article = get_article("Artificial intelligence")

# Display first 1000 characters
print("Article preview:")
print(article[:1000] if len(article) > 1000 else article)
print("...")

# You can also get just the summary
print("\n" + "="*50 + "\n")
print("Summary version:")
summary = wikipedia.summary("Artificial intelligence", sentences=3)
print(summary)

In [ ]:
# Combining Wikipedia with OpenAI for enhanced analysis
def wiki_enhanced_analysis(topic):
    """Use Wikipedia data with OpenAI for comprehensive analysis"""

    # Get Wikipedia information
    wiki_content = get_article(topic)

    # Truncate content if too long (to fit in context window)
    max_chars = 3000
    if len(wiki_content) > max_chars:
        wiki_content = wiki_content[:max_chars] + "..."

    # Use OpenAI to analyze the Wikipedia content
    prompt = f"""Based on the following Wikipedia information about '{topic}':

{wiki_content}

Please provide:
1. A concise summary (2-3 sentences)
2. Three key facts
3. Why this topic is important

Format your response clearly with numbered sections."""

    analysis = ask(prompt, temperature=0.3)
    return analysis

# Example: Analyze a topic using Wikipedia + OpenAI
print("=== Wikipedia-Enhanced Analysis Demo ===\n")
topic = "Artificial intelligence"
print(f"Analyzing: {topic}\n")
result = wiki_enhanced_analysis(topic)
print(result)

### 3.4 Combining Techniques
Using multiple techniques together

In [ ]:
# Complex prompt combining multiple techniques with Wikipedia
def complex_analysis_with_wiki(topic):
    """Analyze a topic using Wikipedia data and multiple prompting techniques"""

    # Try to get Wikipedia information first
    try:
        wiki_summary = wikipedia.summary(topic, sentences=5)
        has_wiki_data = True
    except:
        wiki_summary = "No Wikipedia data available."
        has_wiki_data = False

    # Create a comprehensive prompt
    system_prompt = "You are an expert analyst. Be concise and structured."

    if has_wiki_data:
        user_prompt = f"""Analyze '{topic}' using this format:

Wikipedia Information:
{wiki_summary}

Based on the above information and your knowledge, provide:
1. Current Status (integrate Wikipedia facts with your knowledge)
2. Key Players (list top 3)
3. Future Outlook (brief prediction)

Think step-by-step and provide a structured analysis."""
    else:
        user_prompt = f"""Analyze '{topic}' using this format:

1. Current Status (based on your knowledge)
2. Key Players (list top 3)
3. Future Outlook (brief prediction)

Think step-by-step and provide a structured analysis."""

    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.5
    )

    return response.choices[0].message.content

# Example analysis
topic = "Artificial intelligence"
print(f"Analysis of: {topic}\n")
print(complex_analysis_with_wiki(topic))

## Summary & Best Practices

### Key Takeaways:

1. **Start Simple**: Zero-shot often works well for basic tasks
2. **Add Examples**: Few-shot helps with consistency
3. **Be Specific**: Clear instructions get better results
4. **Think Step-by-Step**: CoT improves reasoning
5. **Format Output**: Structure makes responses more useful
6. **Use External Data**: Wikipedia provides factual information
7. **Control Temperature**: Balance creativity vs consistency

### Quick Reference:
- **Temperature**: 0.0-0.3 (factual), 0.7 (balanced), 1.0+ (creative)
- **System Prompts**: Set consistent behavior
- **User Prompts**: Specific queries
- **Chain-of-Thought**: Add "Think step-by-step"
- **Few-shot**: 2-5 examples usually sufficient
- **Wikipedia**: Use for factual, encyclopedic information

## Practice Exercise

Try creating your own prompt that combines multiple techniques:

In [ ]:
# Your turn! Create a prompt that:
# 1. Uses a role
# 2. Includes few-shot examples
# 3. Requests specific formatting
# 4. Uses chain-of-thought

your_prompt = """
# YOUR PROMPT HERE
"""

# Uncomment to test:
# result = ask(your_prompt)
# print(result)